In [1]:
import os
import numpy as np
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose
from hydra.utils import instantiate
from starfish.data import StarfishDataModule
from starfish.model import FasterRCNNLightning

from pprint import pprint
# Set the environment variable HYDRA_FULL_ERROR=1 to see the full stack trace
os.environ['HYDRA_FULL_ERROR'] = '1'

os.environ['PROJECT_ROOT'] = '/Users/moustholmes/starfishDetection'

# Change the current working directory to the project root
os.chdir('/Users/moustholmes/starfishDetection')

initialize(version_base=None, config_path='configs', job_name='notebook')

cfg_train = compose(config_name='main_config',) #overrides=['experiment=effect_gaussian_nll'])

print(OmegaConf.to_yaml(cfg_train))

# from pytorch_lightning.loggers import WandbLogger
logger = instantiate(cfg_train.logger)

model:
  _target_: starfish.model.FasterRCNNLightning
  num_classes: 2
  learning_rate: 0.001
  momentum: 0.9
  weight_decay: 0.0005
  optimizer:
    _target_: torch.optim.Adam
    _partial_: true
    lr: 0.001
    weight_decay: 0.01
  scheduler:
    _target_: torch.optim.lr_scheduler.StepLR
    _partial_: true
    step_size: 3
    gamma: 0.1
  compile: false
data:
  _target_: starfish.data.StarfishDataModule
  data_path: data/raw
  batch_size: 4
  train_val_test_split:
  - 0.8
  - 0.1
  - 0.1
  subset: 0.002
  num_workers: 8
trainer:
  _target_: pytorch_lightning.Trainer
  min_epochs: 1
  max_epochs: 2
  accelerator: cpu
  devices: 1
logger:
  _target_: pytorch_lightning.loggers.WandbLogger
  offline: false
  id: null
  anonymous: null
  project: CorruptMNIST
  log_model: false
  prefix: ''
  entity: MoustHolmes
  group: ''
  tags:
  - test
  job_type: ''



wandb: Currently logged in as: moustholmes. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# from pytorch_lightning.loggers import WandbLogger
logger = instantiate(cfg_train.logger)

InterpolationResolutionError: ValueError raised while resolving interpolation: HydraConfig was not set

In [2]:
data = instantiate(cfg_train.data)
print(data)
data.setup()
train_dataloader = data.train_dataloader()
batch = next(iter(train_dataloader))
x, y = batch
assert len(x) == data.batch_size
assert len(y) == data.batch_size

Loading 10 images.


In [3]:
model = instantiate(cfg_train.model)
print(model)

FasterRCNNLightning(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
from pytorch_lightning.loggers import WandbLogger
logger = instantiate(cfg_train.logger)

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [5]:
trainer = instantiate(cfg_train.trainer, logger=logger)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [6]:
trainer.fit(model, data)


  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 2/2 [00:47<00:00, 23.90s/it, v_num=pbnx, train_loss_classifier=0.0497, train_loss_box_reg=0.011, train_loss_objectness=2.570, train_loss_rpn_box_reg=0.0146, train_total_loss=2.650, val_AP=0.000]

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('val_AP', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Epoch 1: 100%|██████████| 2/2 [00:58<00:00, 29.36s/it, v_num=pbnx, train_loss_classifier=0.0377, train_loss_box_reg=0.0107, train_loss_objectness=1.950, train_loss_rpn_box_reg=0.055, train_total_loss=2.050, val_AP=0.000]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 2/2 [00:58<00:00, 29.49s/it, v_num=pbnx, train_loss_classifier=0.0377, train_loss_box_reg=0.0107, train_loss_objectness=1.950, train_loss_rpn_box_reg=0.055, train_total_loss=2.050, val_AP=0.000]


In [ ]:
import torchvision
import torch

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')
# For training
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
output = model(images, targets)
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)
# optionally, i